In [2]:
from datetime import datetime, timedelta
import pytz
import pandas as pd
from tqdm import tqdm
from modeler.modeler import Modeler as m
from processor.processor import Processor as p
from database.market import Market
from database.adatabase import ADatabase
import numpy as np

In [3]:
market = Market()
competition_db = ADatabase("competition")

In [12]:
competition_db.connect()
sim = competition_db.retrieve("sim")
competition_db.disconnect()

In [5]:
market.connect()
sp5 = market.retrieve("sp500")
prices = market.retrieve("prices")
market.disconnect()

In [6]:
prices = p.column_date_processing(prices)
prices["year"] = [x.year for x in prices["date"]]
prices["week"] = [x.week for x in prices["date"]]

In [13]:
sim["week"] =  sim["week"] + 1
simulation = prices.merge(sim,on=["year","week","ticker"],how="left")
sim = simulation.dropna()
sim["prediction"] = (sim["xgb_prediction"] + sim["skl_prediction"] + sim["xgb_prediction"]) / 3
sim["delta"] = (sim["prediction"] - sim["adjclose"]) / sim["adjclose"]

In [14]:
sim

,date,close,high,low,open,volume,adjclose,adjhigh,adjlow,adjopen,...,week,training_year,skl_prediction,skl_score,xgb_prediction,xgb_score,cat_prediction,cat_score,prediction,delta
5325,2016-01-11,163.55,170.780,161.800,168.99,9176467,153.938912,160.744038,152.291752,159.059228,...,2,4.0,194.676569,0.985855,192.221542,0.988068,164.684390,0.989783,193.039885,0.254003
5326,2016-01-12,164.01,166.640,163.115,164.78,3460196,154.371880,156.847327,153.529475,155.096631,...,2,4.0,194.676569,0.985855,192.221542,0.988068,164.684390,0.989783,193.039885,0.250486
5327,2016-01-13,157.84,165.960,156.311,163.88,4123521,148.564463,156.207287,147.125315,154.249519,...,2,4.0,194.676569,0.985855,192.221542,0.988068,164.684390,0.989783,193.039885,0.299368
5328,2016-01-14,164.78,166.197,157.550,157.64,3069113,155.096631,156.430360,148.291505,148.376216,...,2,4.0,194.676569,0.985855,192.221542,0.988068,164.684390,0.989783,193.039885,0.244643
5329,2016-01-15,165.00,165.150,158.800,161.52,3422258,155.303702,155.444887,149.468048,152.028206,...,2,4.0,194.676569,0.985855,192.221542,0.988068,164.684390,0.989783,193.039885,0.242983
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2961922,2022-07-29,172.87,176.290,170.990,176.29,1589201,172.870000,176.290000,170.990000,176.290000,...,30,4.0,170.482001,0.974060,171.985840,0.974065,194.973589,0.957151,171.484560,-0.008014
2961923,2022-07-29,34.09,34.340,33.820,34.17,9093603,34.090000,34.340000,33.820000,34.170000,...,30,4.0,30.982478,0.945065,31.903275,0.947003,24.545820,0.900524,31.596342,-0.073149
2961925,2022-07-29,543.53,544.820,519.110,520.00,773745,543.530000,544.820000,519.110000,520.000000,...,30,4.0,474.408920,0.972185,469.768585,0.936435,392.973789,0.956115,471.315363,-0.132862
2961926,2022-07-29,63.48,63.680,61.630,63.30,1781468,63.480000,63.680000,61.630000,63.300000,...,30,4.0,85.049003,0.955982,80.709816,0.948252,99.518389,0.929945,82.156211,0.294206


In [16]:
positions = 10
start_date = datetime(2016,1,1)
end_date = datetime(2022,6,30)
competition_db.connect()
for value in [False]:
    for req in tqdm([0.05,0.1]):
        for position in range(positions):
            date = start_date
            iterration_sim = sim.copy()
            if value == False:
                iterration_sim["delta"] = iterration_sim["delta"] * -1
            while date < end_date:
                try:
                    todays_recs = iterration_sim[(iterration_sim["date"]==date)]
                    todays_recs.sort_values("delta",ascending=False,inplace=True)
                    if todays_recs.index.size > 0 and date.weekday() <= 2:
                        offering = todays_recs.iloc[position]
                        if (offering["delta"] > req):
                            trade = offering
                            ticker = trade["ticker"]
                            buy_price = trade["adjclose"]
                            end_index = 5 - date.weekday()
                            exits = iterration_sim[(iterration_sim["ticker"]==ticker) & (iterration_sim["date"]>date)].iloc[:end_index]
                            exits["gains"] = (exits["adjclose"] - buy_price) / buy_price
                            gain_exits = exits[exits["gains"]>=req].sort_values("date")
                            if gain_exits.index.size < 1:
                                exit = iterration_sim[(iterration_sim["ticker"]==ticker) & (iterration_sim["date"]>date)].iloc[end_index:].iloc[0]
                                trade["sell_price"] = exit["adjclose"]
                            else:
                                exit = gain_exits.iloc[0]
                                trade["sell_price"] = buy_price * (1+(req))
                            delta = (trade["sell_price"] - buy_price) / buy_price
                            trade["sell_date"] = exit["date"]
                            date = exit["date"] + timedelta(days=1)
                            trade["projected_delta"] = offering["delta"]
                            trade["delta"] = delta
                            trade["position"] = position
                            trade["req"] = req
                            trade["value"] = value
                            trade_record = pd.DataFrame([trade]).reset_index(drop=True)[["date","sell_date","ticker","adjclose","sell_price","delta","projected_delta","position","req","value"]]
                            competition_db.store("trades",trade_record)
                        else:
                            date = date + timedelta(days=1)
                    else:
                        date = date + timedelta(days=1)
                except Exception as e:
                    print(str(e))
                    date = date+timedelta(days=1)
competition_db.disconnect()

100%|███████████████████████████████████████████████████████████████████████████████████| 2/2 [11:38<00:00, 349.01s/it]
